# RF-DETR Object Detection Project Notebook
This notebook covers data exploration, visualization, model training, evaluation, and result analysis for the RF-DETR object detection pipeline.

## 1. Setup and Imports

In [ ]:
import os
import json
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from src.data.coco_dataset import COCODetectionDataset
from src.model.detr_rfdetr import RFDETR
from src.model.losses import loss_labels, loss_boxes, loss_giou

## 2. Data Exploration and Visualization

In [ ]:
# Load a split (e.g., train)
data_dir = '../data'
img_dir = os.path.join(data_dir, 'images', 'train')
ann_path = os.path.join(data_dir, 'annotations', 'train.json')
dataset = COCODetectionDataset(img_dir, ann_path)
print(f'Total training images: {len(dataset)}')
# Show a few images with bounding boxes
for i in range(3):
    image, target = dataset[i]
    draw = ImageDraw.Draw(image)
    for box in target['boxes']:
        draw.rectangle(list(box.numpy()), outline='red', width=2)
    plt.imshow(image)
    plt.title(f'Image {i}')
    plt.axis('off')
    plt.show()

## 3. Model Initialization

In [ ]:
num_classes = 3  # WBC, RBC, Platelets
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = RFDETR(num_classes=num_classes).to(device)
print(model)

## 4. Training Loop (Single Epoch Example)

In [ ]:
from torch.utils.data import DataLoader
from torchvision import transforms
def collate_fn(batch):
    return tuple(zip(*batch))
tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_dataset = COCODetectionDataset(img_dir, ann_path, transforms=tfms)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
model.train()
for images, targets in train_loader:
    images = torch.stack(images).to(device)
    # TODO: Implement Hungarian matcher and loss calculation
    outputs = model(images)
    # Placeholder: zero loss
    loss = torch.tensor(0.0, requires_grad=True).to(device)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print('Batch done')
    break  # Remove for full epoch

## 5. Model Evaluation and Visualization

In [ ]:
# Load best model weights if available
model_path = '../rfdetr_best.pth'
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()
# Evaluate on a few test images
test_img_dir = os.path.join(data_dir, 'images', 'test')
test_ann_path = os.path.join(data_dir, 'annotations', 'test.json')
test_dataset = COCODetectionDataset(test_img_dir, test_ann_path, transforms=tfms)
for i in range(3):
    image, target = test_dataset[i]
    with torch.no_grad():
        output = model(image.unsqueeze(0).to(device))
    pred_boxes = output['pred_boxes'].cpu().numpy()[0]
    pred_logits = output['pred_logits'].cpu().numpy()[0]
    scores = pred_logits.max(axis=1)
    labels = pred_logits.argmax(axis=1)
    keep = scores > 0.5
    pred_boxes = pred_boxes[keep]
    labels = labels[keep]
    draw = ImageDraw.Draw(image)
    for box in pred_boxes:
        draw.rectangle(list(box), outline='blue', width=2)
    plt.imshow(image)
    plt.title(f'Test Image {i}')
    plt.axis('off')
    plt.show()

## 6. mAP Calculation (pycocotools)

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
# TODO: Format predictions as COCO results and save to JSON
# pred_json = 'predictions.json'
# ann_file = test_ann_path
# coco_gt = COCO(ann_file)
# coco_dt = coco_gt.loadRes(pred_json)
# coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
# coco_eval.evaluate()
# coco_eval.accumulate()
# coco_eval.summarize()